## ClimaCorePlots

### Constructing a discretization

As we saw in the [Introduction to ClimaCore.jl workshop](../intro-climacore/climacore-api.ipynb), in ClimaCore's API, a spatial discretization is made up of 4 elements:
- `Domain`
- `Mesh`
- `Topology`
- `Space`

In [ ]:
using IntervalSets

import Plots

import ClimaCore
import ClimaCorePlots

### 1D Plots
Let's start with a simple 1D Spectral Element discretization

In [ ]:
domain = ClimaCore.Domains.IntervalDomain(
        ClimaCore.Geometry.XPoint(0.0) .. ClimaCore.Geometry.XPoint(π),
        boundary_tags = (:left, :right),
    )
mesh = ClimaCore.Meshes.IntervalMesh(domain; nelems = 10)
grid_topology = ClimaCore.Topologies.IntervalTopology(mesh)

# Let us set up a Finite Volume discretization as a 0-th order (piecewise constant) Finite Element discretization with Gauss-Legendre quadrature point
fv_quad = ClimaCore.Spaces.Quadratures.GL{1}()
fv_space = ClimaCore.Spaces.SpectralElementSpace1D(grid_topology, fv_quad)

# Let us set up a Spectral Element discretization with 5 Gauss-Legendre-Lobatto nodes (i.e., polynomial degree 4)
sem_quad = ClimaCore.Spaces.Quadratures.GLL{5}()
sem_space = ClimaCore.Spaces.SpectralElementSpace1D(grid_topology, sem_quad)

In [ ]:
# Define the field of coordinates for both discretizations
fv_coords = ClimaCore.Fields.coordinate_field(fv_space)
sem_coords = ClimaCore.Fields.coordinate_field(sem_space)

In [ ]:
# Let's define a simple sinusoidal field on these two spaces

fv_u = sin.(π .* fv_coords.x)

sem_u = sin.(π .* sem_coords.x)

In [ ]:
fv_field_fig = Plots.plot(fv_u)
# Superimpose the second plot, with the bang "!"
sem_and_fv_field_fig = Plots.plot!(sem_u)

In [ ]:
# Let's save the figure in a png file
# First let us set up the file paths
filename = joinpath("./", "1D_fv_and_sem_field.png")
Plots.png(sem_and_fv_field_fig, filename)

### 2D Plots on a non-hybrid plane

In [ ]:
domain = ClimaCore.Domains.RectangleDomain(
        ClimaCore.Geometry.XPoint(0) .. ClimaCore.Geometry.XPoint(2π),
        ClimaCore.Geometry.YPoint(0) .. ClimaCore.Geometry.YPoint(2π),
        x1periodic = true,
        x2periodic = true,
    )

n1, n2 = 2, 2
Nq = 4
mesh = ClimaCore.Meshes.RectilinearMesh(domain, n1, n2)
grid_topology = ClimaCore.Topologies.Topology2D(mesh)
quad = ClimaCore.Spaces.Quadratures.ClosedUniform{Nq + 1}()
rectangle_space = ClimaCore.Spaces.SpectralElementSpace2D(grid_topology, quad)
coords = ClimaCore.Fields.coordinate_field(rectangle_space)

# Let's plot this rectangle space
rectangle_space_fig = Plots.plot(rectangle_space)

In [ ]:
# Let's define a scalar-valued field    
sinxy = map(coords) do coord
    cos(coord.x + coord.y)
end

In [ ]:
rectangle_field_fig = Plots.plot(sinxy)

In [ ]:
rectangle_field_filename = joinpath("./", "2D_rectangle_field.png")
Plots.png(rectangle_field_fig, rectangle_field_filename)

### Hybrid 2D (1DX) Cartesian space

In [ ]:
FT = Float64
helem = 10
velem = 40
npoly = 4

# Let's set up the 1D vertical Finite Difference discretization
vertdomain = ClimaCore.Domains.IntervalDomain(
    ClimaCore.Geometry.ZPoint{FT}(0),
    ClimaCore.Geometry.ZPoint{FT}(1000);
    boundary_tags = (:bottom, :top),
)
vertmesh = ClimaCore.Meshes.IntervalMesh(vertdomain, nelems = velem)
vert_center_space = ClimaCore.Spaces.CenterFiniteDifferenceSpace(vertmesh)

# Let's set up the 2D horizontal Spectral Element discretization
horzdomain = ClimaCore.Domains.IntervalDomain(
    ClimaCore.Geometry.XPoint{FT}(-500) ..
    ClimaCore.Geometry.XPoint{FT}(500),
    periodic = true,
)
horzmesh = ClimaCore.Meshes.IntervalMesh(horzdomain; nelems = helem)
horztopology = ClimaCore.Topologies.IntervalTopology(horzmesh)

quad = ClimaCore.Spaces.Quadratures.GLL{npoly + 1}()
horzspace = ClimaCore.Spaces.SpectralElementSpace1D(horztopology, quad)

# Let's combine these two spaces in an extruded hybrid 1DX space
hv_center_space = ClimaCore.Spaces.ExtrudedFiniteDifferenceSpace(
    horzspace,
    vert_center_space,
)

# Let's define the 2D coordinate field on this hybrid space
coords = ClimaCore.Fields.coordinate_field(hv_center_space)

# Let us plot the X coordinates
xcoords_fig = Plots.plot(coords.x)

In [ ]:
# And the Z coordinates separately
zcoords_fig = Plots.plot(coords.z)

### Hybrid 3D (2DX) Cartesian space

In [ ]:
xelem = 10
yelem = 5
velem = 40
npoly = 4

vertdomain = ClimaCore.Domains.IntervalDomain(
    ClimaCore.Geometry.ZPoint{FT}(0),
    ClimaCore.Geometry.ZPoint{FT}(1000);
    boundary_tags = (:bottom, :top),
)
vertmesh = ClimaCore.Meshes.IntervalMesh(vertdomain, nelems = velem)
vert_center_space = ClimaCore.Spaces.CenterFiniteDifferenceSpace(vertmesh)

xdomain = ClimaCore.Domains.IntervalDomain(
    ClimaCore.Geometry.XPoint{FT}(-500) ..
    ClimaCore.Geometry.XPoint{FT}(500),
    periodic = true,
)
ydomain = ClimaCore.Domains.IntervalDomain(
    ClimaCore.Geometry.YPoint{FT}(-100) ..
    ClimaCore.Geometry.YPoint{FT}(100),
    periodic = true,
)

horzdomain = ClimaCore.Domains.RectangleDomain(xdomain, ydomain)
horzmesh = ClimaCore.Meshes.RectilinearMesh(horzdomain, xelem, yelem)
horztopology = ClimaCore.Topologies.Topology2D(horzmesh)

quad = ClimaCore.Spaces.Quadratures.GLL{npoly + 1}()
horzspace = ClimaCore.Spaces.SpectralElementSpace2D(horztopology, quad)

hv_center_space = ClimaCore.Spaces.ExtrudedFiniteDifferenceSpace(
    horzspace,
    vert_center_space,
)

coords = ClimaCore.Fields.coordinate_field(hv_center_space)

# Let us plot the X coordinates on the XZ plane
xcoords_2dx_fig = Plots.plot(coords.x, slice = (:, 0.0, :))

In [ ]:
# Plot the Y coordinates on the YZ plane
ycoords_2dx_fig = Plots.plot(coords.y, slice = (0.0, :, :))

In [ ]:
# Plot the Z coordinate on the XZ plane
xzcoords_2dx_fig = Plots.plot(coords.z, slice = (:, 0.0, :))

In [ ]:
# Plot the Z coordinate on the YZ plane coordinates
yzcoords_2dx_fig = Plots.plot(coords.z, slice = (0.0, :, :))

In [ ]:
# Let us save all of them

xcoords_2dx_png_filename = joinpath("./", "hybrid_xcoords_center_field.png")
Plots.png(xcoords_2dx_fig, xcoords_2dx_png_filename)

ycoords_2dx_png_filename = joinpath("./", "hybrid_ycoords_center_field.png")
Plots.png(ycoords_2dx_fig, ycoords_2dx_png_filename)

xzcoords_2dx_png_filename = joinpath("./", "hybrid_xzcoords_center_field.png")
Plots.png(xzcoords_2dx_fig, xzcoords_2dx_png_filename)

yzcoords_2dx_png_filename = joinpath("./", "hybrid_yzcoords_center_field.png")
Plots.png(yzcoords_2dx_fig, yzcoords_2dx_png_filename)

### 2D sphere

In [ ]:
R = 6.37122e6

sphere_domain = ClimaCore.Domains.SphereDomain(R)
sphere_mesh = ClimaCore.Meshes.EquiangularCubedSphere(sphere_domain, 8)
sphere_grid_topology = ClimaCore.Topologies.Topology2D(sphere_mesh)
sphere_quad = ClimaCore.Spaces.Quadratures.GLL{4}()
sphere_space = ClimaCore.Spaces.SpectralElementSpace2D(sphere_grid_topology, sphere_quad)
coords = ClimaCore.Fields.coordinate_field(sphere_space)

# Let us define a 2D vector-valued field
u = map(coords) do coord
    u0 = 20.0
    α0 = 45.0
    ϕ = coord.lat
    λ = coord.long

    uu = u0 * (cosd(α0) * cosd(ϕ) + sind(α0) * cosd(λ) * sind(ϕ))
    uv = -u0 * sind(α0) * sind(λ)
    ClimaCore.Geometry.UVVector(uu, uv)
end

In [ ]:
# Let us plot the zonal velocity component
u_field_fig = Plots.plot(u.components.data.:1)

In [ ]:
# And the meridional component of the velocity
v_field_fig = Plots.plot(u.components.data.:2)

In [ ]:
# Let us save them both
u_field_filename = joinpath("./", "2D_cubed_sphere_u_field.png")
Plots.png(u_field_fig, u_field_filename)
v_field_filename = joinpath("./", "2D_cubed_sphere_v_field.png")
Plots.png(v_field_fig, v_field_filename)

### Hybrid 3D sphere

In [ ]:
# Let us reuse the same 2D spectral element space on the sphere and the vertical Finite Difference space we defined earlier
hv_center_space = ClimaCore.Spaces.ExtrudedFiniteDifferenceSpace(sphere_space, vert_center_space)
# And let us define the Face space from the Center space
hv_face_space = ClimaCore.Spaces.FaceExtrudedFiniteDifferenceSpace(hv_center_space)
coords = ClimaCore.Fields.coordinate_field(hv_center_space)

In [ ]:
# Let us define a 3D vector-valued field
u = map(coords) do coord
    u0 = 20.0
    α0 = 45.0
    ϕ = coord.lat
    λ = coord.long
    z = coord.z

    uu = u0 * (cosd(α0) * cosd(ϕ) + sind(α0) * cosd(λ) * sind(ϕ))
    uv = -u0 * sind(α0) * sind(λ)
    uw = z
    ClimaCore.Geometry.UVWVector(uu, uv, uw)
end

# Let us plot the meridional velocity component, at the 3rd vertical level
v_field_level3_fig = Plots.plot(u.components.data.:2, level = 3)

In [ ]:
ClimaCore.level(u.components.data.:3,1)

In [ ]:
w_field_level10_fig = Plots.plot(u.components.data.:3, level = 1, clim=(0,100))